# `TIME inc_ref` doesn't work with a register argument
This is a new problem in tproc v2 revision 18 and the corresponding version of the assembler.

I checked that for these simple example programs, the rev 16 and rev 18 versions of the assembler give the same binary output, so I guess it is not a bug in the assembler, it is either a firmware bug or an update that needs to be made in the assembler.

These firmwares are the same projects as in my regular tprocv2 demos, but recompiled with the rev 18 core, both work with this notebook and show the bug:

* https://s3df.slac.stanford.edu/people/meeg/qick/tprocv2/2024-04-22_216_q3diamond_tprocv2/
* https://s3df.slac.stanford.edu/people/meeg/qick/tprocv2/2024-04-17_111_rfbv2_tprocv2/

I ran this notebook with the 216 firmware and the version of the qick library in awsch/qick-spin.

In [1]:
# jupyter setup boilerplate
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

from qick import *

# for now, all the tProc v2 classes need to be individually imported (can't use qick.*)

# the main program class
from qick.asm_v2 import AveragerProgramV2
# for defining sweeps
from qick.asm_v2 import QickSpan, QickSweep1D

# for low-level tests
from qick.asm_v2 import QickProgramV2

In [2]:
soc = QickSoc(bitfile="/data/fw/2024-04-22_216_q3diamond_tprocv2/qick_216.bit")
soccfg = soc
print(soccfg)


QICK configuration:

	Board: ZCU216

	Software version: 0.2.260
	Firmware timestamp: Mon Apr 22 17:15:55 2024

	Global clocks (MHz): tProcessor 430.080, RF reference 245.760

	10 signal generator channels:
	0:	axis_signal_gen_v6 - envelope memory 65536 samples (9.524 us)
		fs=6881.280 MHz, fabric=430.080 MHz, 32-bit DDS, range=6881.280 MHz
		DAC tile 2, blk 0 is 0_230, on JHC3
	1:	axis_signal_gen_v6 - envelope memory 65536 samples (9.524 us)
		fs=6881.280 MHz, fabric=430.080 MHz, 32-bit DDS, range=6881.280 MHz
		DAC tile 2, blk 1 is 1_230, on JHC4
	2:	axis_sg_mux4_v3 - envelope memory 0 samples (0.000 us)
		fs=6881.280 MHz, fabric=430.080 MHz, 32-bit DDS, range=6881.280 MHz
		DAC tile 2, blk 2 is 2_230, on JHC3
	3:	axis_signal_gen_v6 - envelope memory 65536 samples (9.524 us)
		fs=6881.280 MHz, fabric=430.080 MHz, 32-bit DDS, range=6881.280 MHz
		DAC tile 2, blk 3 is 3_230, on JHC4
	4:	axis_sg_int4_v1 - envelope memory 4096 samples (9.524 us)
		fs=6881.280 MHz, fabric=430.080 MHz, 16-

In [3]:
# from qick.pyro import make_proxy
# # soc, soccfg = make_proxy(ns_host="pynq4x2.dhcp.fnal.gov", ns_port=8888, proxy_name="myqick")
# # soc, soccfg = make_proxy(ns_host="pynq216.dhcp.fnal.gov", ns_port=8888, proxy_name="myqick")
# soc, soccfg = make_proxy(ns_host="pynq111-2.dhcp.fnal.gov", ns_port=8888, proxy_name="myqick")
# print(soccfg)

### using asm_v2
I have a minimal test program which plays a pulse on PMOD0_0 in a loop. There should be 10 pulses, separated by 1 us.

Here are two versions of the program. The first version uses a literal argument to `TIME inc_ref`, and this works. The second version uses a register argument, and this does not work: there is almost no gap between pulses, it's like the reference time did not change.

In [4]:
# this works
reps = 10

prog = QickProgramV2(soccfg)
# initial delay for setup
prog.delay(1.0)
prog.open_loop(reps, "myloop")

prog.trigger(pins=[0], t=0)
prog.delay(1.0)
# prog.delay(QickSweep1D("myloop", 1.0, 1.0))

prog.close_loop()
prog.end()

print("text ASM:")
print(prog.asm())
# print("machine code:")
# prog.compile()
# for a in prog.binprog['pmem']: print([hex(b) for b in a])
# for x in prog.prog_list: print(x)

prog.run(soc)

text ASM:
     TIME ##430 inc_ref 
     REG_WR r0 imm ##10 
MYLOOP:
     REG_WR s14 imm ##0 
     DPORT_WR p0 imm 1 
     REG_WR s14 imm ##10 
     DPORT_WR p0 imm 0 
     TIME ##430 inc_ref 
     REG_WR r0 op -op(r0 - #1) -uf 
     JUMP MYLOOP -if(NZ) 
     JUMP HERE 



In [5]:
# this doesn't work
reps = 10

prog = QickProgramV2(soccfg)
# initial delay for setup
prog.delay(1.0)
prog.open_loop(reps, "myloop")

prog.trigger(pins=[0], t=0)
# prog.delay(1.0)
prog.delay(QickSweep1D("myloop", 1.0, 1.0))

prog.close_loop()
prog.end()

print("text ASM:")
print(prog.asm())
# print("machine code:")
# prog.compile()
# for a in prog.binprog['pmem']: print([hex(b) for b in a])
# for x in prog.prog_list: print(x)

prog.run(soc)

text ASM:
     REG_WR r1 imm ##430 
     TIME ##430 inc_ref 
     REG_WR r0 imm ##10 
MYLOOP:
     REG_WR s14 imm ##0 
     DPORT_WR p0 imm 1 
     REG_WR s14 imm ##10 
     DPORT_WR p0 imm 0 
     TIME inc_ref r1 
     REG_WR r0 op -op(r0 - #1) -uf 
     JUMP MYLOOP -if(NZ) 
     JUMP HERE 



### only using the assembler
Just to make sure, I do the same test with the same two programs, but using the text ASM and the assembler, nothing else. Same results.

In [6]:
# this works
from qick.tprocv2_assembler import Assembler
pstr = """
     TIME #430 inc_ref 
     REG_WR r0 imm #10 
MYLOOP:
     REG_WR s14 imm #0 
     DPORT_WR p0 imm 1 
     REG_WR s14 imm #10 
     DPORT_WR p0 imm 0 
     TIME #430 inc_ref 
     REG_WR r0 op -op(r0 - #1) -uf 
     JUMP MYLOOP -if(NZ) 
     .END"""
print("text ASM:")
print(pstr)

# # convert text ASM to machine code
# print("\nmachine code:")
# for a in Assembler.str_asm2bin(pstr)[1]:
#     print([hex(b) for b in a])

# # convert text ASM to ASM dicts
# print("\nASM dicts:")
# plist,labels = Assembler.str_asm2list(pstr)
# for a in plist:
#     print(a)
# for a in labels.items():
#     print(a)

# # convert ASM dicts back to text ASM
# print("\ntext ASM -> ASM dicts -> text ASM:")
# print(Assembler.list2asm(plist, labels))

binprog = {'pmem': Assembler.str_asm2bin(pstr)[1], 'wmem': np.zeros((0,8), dtype=np.int32)}
soc.load_bin_program(binprog)
soc.start_tproc()

text ASM:

     TIME #430 inc_ref 
     REG_WR r0 imm #10 
MYLOOP:
     REG_WR s14 imm #0 
     DPORT_WR p0 imm 1 
     REG_WR s14 imm #10 
     DPORT_WR p0 imm 0 
     TIME #430 inc_ref 
     REG_WR r0 op -op(r0 - #1) -uf 
     JUMP MYLOOP -if(NZ) 
     .END


In [7]:
# this doesn't work
from qick.tprocv2_assembler import Assembler
pstr = """
     REG_WR r1 imm #430
     TIME #430 inc_ref
     REG_WR r0 imm #10
MYLOOP:
     REG_WR s14 imm #0
     DPORT_WR p0 imm 1
     REG_WR s14 imm #10
     DPORT_WR p0 imm 0
     TIME inc_ref r1
     REG_WR r0 op -op(r0 - #1) -uf
     JUMP MYLOOP -if(NZ)
     .END"""
print("text ASM:")
print(pstr)

# # convert text ASM to machine code
# print("\nmachine code:")
# for a in Assembler.str_asm2bin(pstr)[1]:
#     print([hex(b) for b in a])

# # convert text ASM to ASM dicts
# print("\nASM dicts:")
# plist,labels = Assembler.str_asm2list(pstr)
# for a in plist:
#     print(a)
# for a in labels.items():
#     print(a)

# # convert ASM dicts back to text ASM
# print("\ntext ASM -> ASM dicts -> text ASM:")
# print(Assembler.list2asm(plist, labels))

binprog = {'pmem': Assembler.str_asm2bin(pstr)[1], 'wmem': np.zeros((0,8), dtype=np.int32)}
soc.load_bin_program(binprog)
soc.start_tproc()

text ASM:

     REG_WR r1 imm #430
     TIME #430 inc_ref
     REG_WR r0 imm #10
MYLOOP:
     REG_WR s14 imm #0
     DPORT_WR p0 imm 1
     REG_WR s14 imm #10
     DPORT_WR p0 imm 0
     TIME inc_ref r1
     REG_WR r0 op -op(r0 - #1) -uf
     JUMP MYLOOP -if(NZ)
     .END


### simplify
Unroll the loop and use immediate times in `DPORT_WR`, just play 5 pulses. Same behavior.

In [8]:
# works
from qick.tprocv2_assembler import Assembler
pstr = """
     TIME inc_ref #430
     DPORT_WR p0 imm 1 @0
     DPORT_WR p0 imm 0 @10
     TIME inc_ref #430
     DPORT_WR p0 imm 1 @0
     DPORT_WR p0 imm 0 @10
     TIME inc_ref #430
     DPORT_WR p0 imm 1 @0
     DPORT_WR p0 imm 0 @10
     TIME inc_ref #430
     DPORT_WR p0 imm 1 @0
     DPORT_WR p0 imm 0 @10
     TIME inc_ref #430
     DPORT_WR p0 imm 1 @0
     DPORT_WR p0 imm 0 @10
     TIME inc_ref #430
     .END"""
print("text ASM:")
print(pstr)

binprog = {'pmem': Assembler.str_asm2bin(pstr)[1], 'wmem': np.zeros((0,8), dtype=np.int32)}
soc.load_bin_program(binprog)
soc.start_tproc()

text ASM:

     TIME inc_ref #430
     DPORT_WR p0 imm 1 @0
     DPORT_WR p0 imm 0 @10
     TIME inc_ref #430
     DPORT_WR p0 imm 1 @0
     DPORT_WR p0 imm 0 @10
     TIME inc_ref #430
     DPORT_WR p0 imm 1 @0
     DPORT_WR p0 imm 0 @10
     TIME inc_ref #430
     DPORT_WR p0 imm 1 @0
     DPORT_WR p0 imm 0 @10
     TIME inc_ref #430
     DPORT_WR p0 imm 1 @0
     DPORT_WR p0 imm 0 @10
     TIME inc_ref #430
     .END


In [9]:
# doesn't work
from qick.tprocv2_assembler import Assembler
pstr = """
     REG_WR r1 imm #430
     TIME inc_ref #430
     DPORT_WR p0 imm 1 @0
     DPORT_WR p0 imm 0 @10
     TIME inc_ref r1
     DPORT_WR p0 imm 1 @0
     DPORT_WR p0 imm 0 @10
     TIME inc_ref r1
     DPORT_WR p0 imm 1 @0
     DPORT_WR p0 imm 0 @10
     TIME inc_ref r1
     DPORT_WR p0 imm 1 @0
     DPORT_WR p0 imm 0 @10
     TIME inc_ref r1
     DPORT_WR p0 imm 1 @0
     DPORT_WR p0 imm 0 @10
     TIME inc_ref r1
     .END"""
print("text ASM:")
print(pstr)

binprog = {'pmem': Assembler.str_asm2bin(pstr)[1], 'wmem': np.zeros((0,8), dtype=np.int32)}
soc.load_bin_program(binprog)
soc.start_tproc()

text ASM:

     REG_WR r1 imm #430
     TIME inc_ref #430
     DPORT_WR p0 imm 1 @0
     DPORT_WR p0 imm 0 @10
     TIME inc_ref r1
     DPORT_WR p0 imm 1 @0
     DPORT_WR p0 imm 0 @10
     TIME inc_ref r1
     DPORT_WR p0 imm 1 @0
     DPORT_WR p0 imm 0 @10
     TIME inc_ref r1
     DPORT_WR p0 imm 1 @0
     DPORT_WR p0 imm 0 @10
     TIME inc_ref r1
     DPORT_WR p0 imm 1 @0
     DPORT_WR p0 imm 0 @10
     TIME inc_ref r1
     .END
